# Functions

In [33]:
import numpy as np
import scipy as scp
import math

def scale_factor(z):
    return 1/(1+z)

z = 0.1
a = scale_factor(z)
H0 = 75
omega_DE = 0.669
omega_M = 1 - omega_DE
c = 299792.458

def Friedmann_eq(z):
    a = scale_factor(z)
    return H0*np.sqrt(omega_M/(a**3) + omega_DE + (1-omega_M-omega_DE)/(a**2))

def integrand(z):
    return c/Friedmann_eq(z)

def co_moving_dist(z_up):
    dist, err = scp.integrate.quad(integrand, 0, z_up)
    return dist

def co_moving_dist_near(z):
    return z*c/H0

rc = co_moving_dist(z)
print(rc)

def flux_model(rc, Lp, z):
    flux = Lp/(4*np.pi*(rc**2)*((1+z)*2))
#then convert this to magnitudes and compare with the data

#use this to find Lp using chi-sq minimisation
def flux_model_near(Lp, z):
    rc = co_moving_dist_near(z)
    flux = Lp/(4*np.pi*(rc)**2*((1+z)**2))
    return flux

389.6576938798581


# Load the data

In [30]:
sn_z_near, sn_mag_near, sn_mag_err_near =  np.loadtxt("sn_data_nearby.txt", unpack = True, usecols=(1,2,3))
#sn_z_near = np.array(sn_z_near)
print(sn_z_near)


[0.03  0.05  0.026 0.075 0.026 0.014 0.101 0.02  0.036 0.045 0.043 0.018
 0.079 0.088 0.063 0.071 0.052 0.05 ]


# Finding peak luminosity:

In [50]:
f0 = 1
#6.61 * 10**(-9)
# * 10**(-7) * (0.01)**(-2) * (10**(-10))**(-1)

def mag_model_near(z, *Lpl):
    rc = co_moving_dist_near(z)
    flux = Lpl[0]/(4*np.pi*(rc)**2)
    argument = flux/f0
    print(argument)
    mag = -2.5*np.log10(argument)
    if mag.any() == 'nan':
        print("Error!")
    return mag

# Chi squared mininimisation to find peak luminosity:

In [52]:
initial = np.array([100]) #initial guess for L_peak_lambda
deg_freedom = sn_z_near.size - initial.size #change x to actual data values
popt, cov = scp.optimize.curve_fit(mag_model_near, # function to fit
                                     sn_z_near, # x data
                                     sn_mag_near, # y data
                                     sigma=sn_mag_err_near, # set yerr as the array of error bars for the fit
                                     absolute_sigma=True, # errors bars DO represent 1 std error
                                     p0=initial, # starting point for fit
                                     check_finite=True)
# define variables
def chisq(model_params, x_data, y_data, y_err):
    chisqval=0
    for i in range(len(x_data)):
        chisqval += ((y_data[i] - mag_model_near(x_data[i], *model_params))/y_err[i])**2 
        # NOTE again the asterisk (*) before 'model_params' here!
    return chisqval

chisq_min = chisq(popt, sn_z_near, sn_mag_near, sn_mag_err_near)
reduced_chisq_min = chisq_min/deg_freedom
print(reduced_chisq_min)
popt_errs = np.sqrt(np.diag(cov))
for i in range(len(popt)):
    print('optimised parameter[{}] = {} +/- {}'.format(i, popt[i], popt_errs[i]))

[5.53386739e-04 1.99219226e-04 7.36757492e-04 8.85418782e-05
 7.36757492e-04 2.54106155e-03 4.88234550e-05 1.24512016e-03
 3.84296346e-04 2.45949662e-04 2.69360771e-04 1.53718539e-03
 7.98026061e-05 6.43140579e-05 1.25484521e-04 9.87994574e-05
 1.84189373e-04 1.99219226e-04]
[5.53386747e-04 1.99219229e-04 7.36757503e-04 8.85418795e-05
 7.36757503e-04 2.54106159e-03 4.88234558e-05 1.24512018e-03
 3.84296352e-04 2.45949665e-04 2.69360775e-04 1.53718541e-03
 7.98026073e-05 6.43140589e-05 1.25484523e-04 9.87994589e-05
 1.84189376e-04 1.99219229e-04]
[-0.00368731 -0.00132743 -0.00490914 -0.00058997 -0.00490914 -0.01693153
 -0.00032532 -0.00829645 -0.00256063 -0.00163881 -0.0017948  -0.01024253
 -0.00053174 -0.00042854 -0.00083613 -0.00065832 -0.00122729 -0.00132743]
[-8.59829898e-04 -3.09538763e-04 -1.14474395e-03 -1.37572784e-04
 -1.14474395e-03 -3.94819851e-03 -7.58599067e-05 -1.93461727e-03
 -5.97104096e-04 -3.82146621e-04 -4.18521854e-04 -2.38841638e-03
 -1.23994057e-04 -9.99285780e-05 

C:\Users\milan\AppData\Local\Temp\ipykernel_13300\105964205.py:10: RuntimeWarning: invalid value encountered in log10
  mag = -2.5*np.log10(argument)
